# Imports

In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

import pickle

from factor_analyzer import FactorAnalyzer
from sklearn.cluster import KMeans

In [7]:
import statsmodels.api as sm
from statsmodels.formula.api import glm
from statsmodels.genmod.families import Poisson, NegativeBinomial, Gaussian, Gamma
from scipy.stats import genextreme as gev
from scipy.stats import beta, norm, gamma, weibull_min

In [8]:
import torch
import pyro

import pyro.distributions as dist

from pyro.optim import Adam
from pyro.contrib.autoguide import AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO, Predictive


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Intro

In this notebook, all parameters for the simulation model are estimated from the datasets

The parameters that will be estimated in this notebook are the following:
- Factor model / K-means clustering
    - Floods
    - Storms
- Proportion of Outliers
    - Floods
    - Storms
- Stationary Magnitudes and Durations
    - Flood Durations, outliers
    - Flood Durations, non-outliers
    - Storm Durations, non-outliers
    - Storm Magnitudes, all
- Trending Magnitudes and Durations
    - Flood Magnitudes, outliers
    - Flood Magnitudes, non-outliers
    - Storm Durations, outliers

# Data

In [10]:
#########################
# LOAD DISASTER DATA
#########################

emdat = pd.read_excel("data/emdat_public.xlsx")
emdat = emdat[emdat["Year"] >= 1800]
emdat = emdat[emdat["Year"] <= 2020]

# Filling NaNs in start month and day with 1
emdat['Start Month'].fillna(1, inplace=True)
emdat['Start Day'].fillna(1, inplace=True)

# Filling NaNs in end month and day with corresponding start month and day
emdat['End Month'] = emdat.apply(lambda row: row['Start Month'] if pd.isna(row['End Month']) else row['End Month'], axis=1)
emdat['End Day'] = emdat.apply(lambda row: row['Start Day'] if pd.isna(row['End Day']) else row['End Day'], axis=1)

### DURATION ###
# Convert 'Start' and 'End' columns to datetime
emdat['Start Date'] = pd.to_datetime(emdat[['Start Year', 'Start Month', 'Start Day']]
                                  .rename(columns={'Start Year': 'year', 'Start Month': 'month', 'Start Day': 'day'}))

emdat['End Date'] = pd.to_datetime(emdat[['End Year', 'End Month', 'End Day']]
                                .rename(columns={'End Year': 'year', 'End Month': 'month', 'End Day': 'day'}))


# Calculate the 'Duration' column as the difference in days between 'End Date' and 'Start Date'
emdat['Duration'] = (emdat['End Date'] - emdat['Start Date']).dt.days

# Set all negative values in DataFrame to 0
emdat["Duration"] = emdat["Duration"].clip(lower=0)

# Replace east and west germany with DEU
emdat["ISO"] = emdat["ISO"].replace("DFR","DEU")
emdat["ISO"] = emdat["ISO"].replace("DDR","DEU")

## Correct more codes
emdat["ISO"] = emdat["ISO"].replace("YMD","YEM") # Replace old code for Yemen with the new one
emdat["ISO"] = emdat["ISO"].replace("YMN","YEM") # Replace the other old code for Yemen with the new one
emdat["ISO"] = emdat["ISO"].replace("SUN","RUS") # Replace Soviet Union with Russia
emdat["ISO"] = emdat["ISO"].replace("AZO","PRT") # Replace the Azores with Portugal
emdat["ISO"] = emdat["ISO"].replace("SSD","SDN") # Replace South Sudan with Sudan

emdat = emdat[emdat["ISO"] != "PRK"] #Remove North Korea
emdat = emdat[emdat["ISO"] != "YUG"] #Remove Yugoslavia
emdat = emdat[emdat["ISO"] != "CSK"] #Remove Chechoslovakia
emdat = emdat[emdat["ISO"] != "REU"] #Remove Reunion
emdat = emdat[emdat["ISO"] != "SCG"] #Remove Serbia and Montenegro
emdat = emdat[emdat["ISO"] != "SYC"] #Remove Seychelles
emdat = emdat[emdat["ISO"] != "SPI"] #Remove Canary Islands
emdat = emdat[emdat["ISO"] != "SHN"] #Remove Saint Helena
emdat = emdat[emdat["ISO"] != "IMN"] #Remove Isle of Man

# Problem with storm windspeeds
emdat = emdat.drop(emdat[(emdat["Disaster Type"] == "Storm") & (emdat["Dis Mag Value"] > 408)].index)

#Problem with storm durations
emdat = emdat.drop(emdat[(emdat["Disaster Type"] == "Storm") & (emdat["Duration"] > 23)].index)


# Create disaster dataframes for floods and storms that cause displacement
dis_df = emdat[emdat["Year"] >= 2000]
dis_df = dis_df[dis_df["Year"] <= 2022]
dis_df["Duration"] = dis_df["Duration"].clip(lower=1)
dis_df["Dis Mag Value"] = dis_df["Dis Mag Value"].clip(lower=0)
dis_df_f = dis_df[dis_df["Disaster Type"] == "Flood"]
dis_df_s = dis_df[dis_df["Disaster Type"] == "Storm"]
dis_df_s['Disp'] = dis_df_s['No Homeless'].apply(lambda x: 1 if x > 0 else 0).astype(int)
dis_df_f['Disp'] = dis_df_f['No Homeless'].apply(lambda x: 1 if x > 0 else 0).astype(int)

/tmp/ipykernel_3665/348924813.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_df_s['Disp'] = dis_df_s['No Homeless'].apply(lambda x: 1 if x > 0 else 0).astype(int)
/tmp/ipykernel_3665/348924813.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dis_df_f['Disp'] = dis_df_f['No Homeless'].apply(lambda x: 1 if x > 0 else 0).astype(int)


In [11]:
#########################
# LOAD WORLD RISK INDEX DATA
#########################

wri = pd.read_excel("data/WRI_FullData_Time-series-2000-2022.xlsx", sheet_name=None)
del wri["Codes"]

data = []
for k,v in wri.items():
    year = k.split()[-1]
    vals = v.copy()
    #Removing all composite KPI's
    vals = vals[["Country", "Code",
                 "EI_02b", "EI_02d", "EI_02f", "EI_03b", "EI_03d", "EI_03f", "EI_05b", "EI_05d", "EI_05f", "EI_07b",
                 "SI_01a", "SI_02b", "SI_02a", "SI_03a", "SI_05a", "SI_08a", "SI_12b", "SI_13b", 
                 "CI_01b", "CI_05b", 
                 "AI_01a", "AI_02a"]]
    vals["Year"] = year

    data.append(vals)
    #wri_df = wri_df.merge(v, on="Country", how="inner")
wri_df = pd.concat(data, ignore_index=True)
wri_df["Year"] = wri_df["Year"].astype(int)
wri_df.rename(columns={"Code":"ISO"},inplace=True)

In [12]:
#########################
# LOAD POPULATION DENSITY DATA
#########################

pop = pd.read_csv("data/country_pop.csv")
pop = pop.drop(["Country Name", "Indicator Name", "Indicator Code"],axis=1)
pop = pd.melt(pop, id_vars=['Country Code'], var_name='Year', value_name='Value')
pop.rename(columns={"Country Code":"ISO", "Value":"Pop"} ,inplace=True)
pop = pop.dropna()
size = pd.read_csv("data/country_size.csv")
size = size.drop(["Country Name", "Indicator Name", "Indicator Code"],axis=1)
size = pd.melt(size, id_vars=['Country Code'], var_name='Year', value_name='Value')
size.rename(columns={"Country Code":"ISO", "Value":"Size"},inplace=True)
size = size.dropna()
popsize = pd.merge(size,pop, on=["ISO","Year"],how="left")
popsize["Year"] = popsize["Year"].astype(int)
popsize["Popdens"] = popsize["Pop"]/popsize["Size"]

In [13]:
#########################
# LOAD GLOBAL TEMPERATURE ANOMALY DATA
#########################

temp = pd.read_csv("data/env_params/SSP119/global_params.csv")
temp = temp.drop("co2_emissions", axis=1)
temp.rename(columns={"time":"Year"},inplace=True)

# Factor Model / K-means cluster

Storm factor analysis

In [14]:
# Storm factor analysis

X_s = pd.merge(dis_df_s, wri_df, on=["ISO", "Year"],how="left")
X_s = pd.merge(X_s,popsize, on=["Year","ISO"],how="left")
X_s = pd.merge(X_s,temp, on="Year", how="left")
X_s["Homeless_pct"] = X_s["No Homeless"] / X_s["Pop"]
X_s["Year"] = X_s["Year"] - 2000
#X = pd.get_dummies(X, columns=['Region'])
# Convert all boolean columns in the DataFrame to integers (0 or 1)
bool_columns = X_s.select_dtypes(include=['bool']).columns
X_s[bool_columns] = X_s[bool_columns].astype(int)

# REMOVE REDUNDANT VARIABLES
# We remove all indicators that are in numbers and stick to the ones in percentages, as they are otherwise the same
X_s = X_s.drop(["Disaster Type", "Dis Mag Scale","Country_y", "ISO", "Region", "Country_x","No Affected","No Homeless", "Start Date", "End Date", "Size", "Pop",
                "Dis No", "Seq", "Glide", "Disaster Group", "Disaster Subgroup", "Disaster Subtype", "Disaster Subsubtype", "Event Name", "Continent",
                "Location", "Origin", "Associated Dis", "Associated Dis2", "OFDA Response", "Appeal", "Declaration", "AID Contribution ('000 US$)",
                "River Basin", "Start Year", "Start Month", "Start Day", "End Year", "End Month", "End Day", "Latitude", "Longitude", "Local Time",
                "Total Deaths", "No Injured", "Total Affected", "Reconstruction Costs ('000 US$)", "Reconstruction Costs, Adjusted ('000 US$)",
                "Insured Damages ('000 US$)", "Insured Damages, Adjusted ('000 US$)", "Total Damages ('000 US$)", "Total Damages, Adjusted ('000 US$)",
                "CPI", "Adm Level", "Admin1 Code", "Admin2 Code", "Geo Locations", "T"],axis=1)
X_s = X_s.drop("Homeless_pct", axis=1)
X_s=X_s.dropna()
y_s = X_s["Disp"]
X_s = X_s.drop("Disp",axis=1)

In [15]:
X_s

,Year,Dis Mag Value,Duration,EI_02b,EI_02d,EI_02f,EI_03b,EI_03d,EI_03f,EI_05b,...,SI_03a,SI_05a,SI_08a,SI_12b,SI_13b,CI_01b,CI_05b,AI_01a,AI_02a,Popdens
1,0,70.0,23,29.66,40.37,46.70,47.42,49.49,54.90,49.40,...,68.32,27.84,54.59,21.58,65.08,62.67,89.84,69.82,65.99,992.496942
2,0,160.0,5,38.90,38.36,43.77,69.44,69.29,71.86,60.82,...,56.90,44.38,42.39,21.58,55.86,75.66,61.13,63.06,60.66,134.492481
5,0,120.0,1,29.66,40.37,46.70,47.42,49.49,54.90,49.40,...,68.32,27.84,54.59,21.58,65.08,62.67,89.84,69.82,65.99,992.496942
6,0,80.0,1,29.66,40.37,46.70,47.42,49.49,54.90,49.40,...,68.32,27.84,54.59,21.58,65.08,62.67,89.84,69.82,65.99,992.496942
7,0,100.0,1,29.66,40.37,46.70,47.42,49.49,54.90,49.40,...,68.32,27.84,54.59,21.58,65.08,62.67,89.84,69.82,65.99,992.496942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,20,55.0,1,56.90,56.21,55.44,46.87,45.01,45.13,82.67,...,54.70,38.03,38.58,29.87,77.46,60.47,42.08,56.65,63.05,376.265141
1299,20,155.0,1,56.90,56.21,55.44,46.87,45.01,45.13,82.67,...,54.70,38.03,38.58,29.87,77.46,60.47,42.08,56.65,63.05,376.265141
1306,20,75.0,1,41.46,38.50,31.74,89.46,89.40,83.38,60.82,...,55.91,32.15,28.48,18.07,63.10,48.38,59.49,81.55,13.09,308.359102
1307,20,85.0,1,41.46,38.50,31.74,89.46,89.40,83.38,60.82,...,55.91,32.15,28.48,18.07,63.10,48.38,59.49,81.55,13.09,308.359102


In [16]:
scaler_s = StandardScaler()
X_scaled_s = scaler_s.fit_transform(X_s)

# Step 2: Perform Factor Analysis
fa_s = FactorAnalyzer(n_factors=2, rotation='varimax')
fa_s.fit(X_scaled_s)

# Calculate factor scores
X_fa_s = fa_s.transform(X_scaled_s)

# Get the loadings (the rotated factor loadings matrix)
loadings = fa_s.loadings_

num_clusters = 3

# Use a clustering algorithm to find clusters in the factor scores
kmeans_s = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans_s.fit_predict(X_fa_s)

# Calculate the probability of each label within each cluster
label_probs_s = np.zeros((num_clusters, 2))
for i in range(num_clusters):
    cluster_indices = np.where(clusters == i)[0]
    # Get the labels for each cluster using .iloc for correct indexing
    cluster_labels = y_s.iloc[cluster_indices]
    label_probs_s[i, 0] = np.mean(cluster_labels == 0)
    label_probs_s[i, 1] = np.mean(cluster_labels == 1)

#params["storm_fa_loadings"] = loadings
#params["storm_kmeans"] = 
# Save the scaler, factor analyzer, and k-means model
with open('data/model_params/scaler_storm.pkl', 'wb') as file:
    pickle.dump(scaler_s, file)
with open('data/model_params/factor_analyzer_storm.pkl', 'wb') as file:
    pickle.dump(fa_s, file)
with open('data/model_params/kmeans_storm.pkl', 'wb') as file:
    pickle.dump(kmeans_s, file)
with open('data/model_params/label_probs_storms.pkl', 'wb') as file:
    pickle.dump(label_probs_s, file)


/home/jpoulsen/miniconda3/envs/abm/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Flood

In [17]:
X_f = pd.merge(dis_df_f, wri_df, on=["ISO", "Year"],how="left")
X_f = pd.merge(X_f,popsize, on=["Year","ISO"],how="left")
X_f = pd.merge(X_f,temp, on="Year", how="left")
X_f["Homeless_pct"] = X_f["No Homeless"] / X_f["Pop"]
X_f["Year"] = X_f["Year"] - 2000
#X = pd.get_dummies(X, columns=['Region'])
# Convert all boolean columns in the DataFrame to integers (0 or 1)
bool_columns = X_f.select_dtypes(include=['bool']).columns
X_f[bool_columns] = X_f[bool_columns].astype(int)

# REMOVE REDUNDANT VARIABLES
# We remove all indicators that are in numbers and stick to the ones in percentages, as they are otherwise the same
X_f = X_f.drop(["Disaster Type", "Dis Mag Scale","Country_y", "ISO", "Region", "Country_x","No Affected","No Homeless", "Start Date", "End Date", "Size", "Pop",
                "Dis No", "Seq", "Glide", "Disaster Group", "Disaster Subgroup", "Disaster Subtype", "Disaster Subsubtype", "Event Name", "Continent",
                "Location", "Origin", "Associated Dis", "Associated Dis2", "OFDA Response", "Appeal", "Declaration", "AID Contribution ('000 US$)",
                "River Basin", "Start Year", "Start Month", "Start Day", "End Year", "End Month", "End Day", "Latitude", "Longitude", "Local Time",
                "Total Deaths", "No Injured", "Total Affected", "Reconstruction Costs ('000 US$)", "Reconstruction Costs, Adjusted ('000 US$)",
                "Insured Damages ('000 US$)", "Insured Damages, Adjusted ('000 US$)", "Total Damages ('000 US$)", "Total Damages, Adjusted ('000 US$)",
                "CPI", "Adm Level", "Admin1 Code", "Admin2 Code", "Geo Locations", "T"],axis=1)
X_f = X_f.drop("Homeless_pct", axis=1)
X_f=X_f.dropna()
y_f = X_f["Disp"]
X_f = X_f.drop("Disp",axis=1)

In [18]:
scaler_f = StandardScaler()
X_scaled_f = scaler_f.fit_transform(X_f)

# Step 2: Perform Factor Analysis
fa_f = FactorAnalyzer(n_factors=2, rotation='varimax')
fa_f.fit(X_scaled_f)

# Calculate factor scores
X_fa_f = fa_f.transform(X_scaled_f)

# Get the loadings (the rotated factor loadings matrix)
loadings = fa_f.loadings_

num_clusters = 3

# Use a clustering algorithm to find clusters in the factor scores
kmeans_f = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans_f.fit_predict(X_fa_f)

# Calculate the probability of each label within each cluster
label_probs_f = np.zeros((num_clusters,2))
for i in range(num_clusters):
    cluster_indices = np.where(clusters == i)[0]
    # Get the labels for each cluster using .iloc for correct indexing
    cluster_labels = y_f.iloc[cluster_indices]
    label_probs_f[i, 0] = np.mean(cluster_labels == 0)
    label_probs_f[i, 1] = np.mean(cluster_labels == 1)

# Save the scaler, factor analyzer, and k-means model
with open('data/model_params/scaler_flood.pkl', 'wb') as file:
    pickle.dump(scaler_f, file)
with open('data/model_params/factor_analyzer_flood.pkl', 'wb') as file:
    pickle.dump(fa_f, file)
with open('data/model_params/kmeans_flood.pkl', 'wb') as file:
    pickle.dump(kmeans_f, file)
with open('data/model_params/label_probs_floods.pkl', 'wb') as file:
    pickle.dump(label_probs_f, file)


/home/jpoulsen/miniconda3/envs/abm/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


# Proportion of Outliers

In [19]:
def outlier_seperation(df, type, var):
    emdat_filter = df[df["Disaster Type"] == type]
    emdat_filter = emdat_filter[emdat_filter["Year"] > 1961]
    # Calculate Q1, Q3, and IQR for each year
    Q1 = emdat_filter.groupby('Year')[var].quantile(0.25)
    Q3 = emdat_filter.groupby('Year')[var].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Count outliers for each year
    outliers = emdat_filter.apply(lambda x: (x[var] > upper_bound[x['Year']]), axis=1)

    # Creating a DataFrame for outliers
    outliers_df = emdat_filter[outliers]

    # Creating a DataFrame for non-outliers
    non_outliers_df = emdat_filter[~outliers]

    return outliers_df, non_outliers_df

In [20]:
#########################
# LOAD GLOBAL TEMPERATURE ANOMALY DATA
#########################

temp = pd.read_csv("data/env_params/SSP119/global_params.csv")
temp = temp.drop("co2_emissions", axis=1)
temp.rename(columns={"time":"Year"},inplace=True)

In [21]:
#########################
# LOAD DISASTER DATA
#########################

emdat = pd.read_excel("data/emdat_public.xlsx")
emdat = emdat[emdat["Year"] >= 1800]
emdat = emdat[emdat["Year"] <= 2020]

# Limit the data to only floods and storms
disaster_types = ["Flood", "Storm"]
emdat = emdat[emdat["Disaster Type"].isin(disaster_types)]

# Filling NaNs in start month and day with 1
emdat['Start Month'].fillna(1, inplace=True)
emdat['Start Day'].fillna(1, inplace=True)

# Filling NaNs in end month and day with corresponding start month and day
emdat['End Month'] = emdat.apply(lambda row: row['Start Month'] if pd.isna(row['End Month']) else row['End Month'], axis=1)
emdat['End Day'] = emdat.apply(lambda row: row['Start Day'] if pd.isna(row['End Day']) else row['End Day'], axis=1)

# Problem with storm windspeeds
emdat = emdat.drop(emdat[(emdat["Disaster Type"] == "Storm") & (emdat["Dis Mag Value"] > 408)].index)

### DURATION ###
# Convert 'Start' and 'End' columns to datetime
emdat['Start Date'] = pd.to_datetime(emdat[['Start Year', 'Start Month', 'Start Day']]
                                  .rename(columns={'Start Year': 'year', 'Start Month': 'month', 'Start Day': 'day'}))

emdat['End Date'] = pd.to_datetime(emdat[['End Year', 'End Month', 'End Day']]
                                .rename(columns={'End Year': 'year', 'End Month': 'month', 'End Day': 'day'}))


# Calculate the 'Duration' column as the difference in days between 'End Date' and 'Start Date'
emdat['Duration'] = (emdat['End Date'] - emdat['Start Date']).dt.days

# Set all negative values in DataFrame to 0
emdat["Duration"] = emdat["Duration"].clip(lower=0)

#Problem with storm durations
#emdat = emdat.drop(emdat[(emdat["Disaster Type"] == "Storm") & (emdat["Duration"] > 23)].index)

# Replace east and west germany with DEU
emdat["ISO"] = emdat["ISO"].replace("DFR","DEU")
emdat["ISO"] = emdat["ISO"].replace("DDR","DEU")

## Correct more codes
emdat["ISO"] = emdat["ISO"].replace("YMD","YEM") # Replace old code for Yemen with the new one
emdat["ISO"] = emdat["ISO"].replace("YMN","YEM") # Replace the other old code for Yemen with the new one
emdat["ISO"] = emdat["ISO"].replace("SUN","RUS") # Replace Soviet Union with Russia
emdat["ISO"] = emdat["ISO"].replace("AZO","PRT") # Replace the Azores with Portugal
emdat["ISO"] = emdat["ISO"].replace("SSD","SDN") # Replace South Sudan with Sudan

emdat = emdat[emdat["ISO"] != "PRK"] #Remove North Korea
emdat = emdat[emdat["ISO"] != "YUG"] #Remove Yugoslavia
emdat = emdat[emdat["ISO"] != "CSK"] #Remove Chechoslovakia
emdat = emdat[emdat["ISO"] != "REU"] #Remove Reunion
emdat = emdat[emdat["ISO"] != "SCG"] #Remove Serbia and Montenegro
emdat = emdat[emdat["ISO"] != "SYC"] #Remove Seychelles
emdat = emdat[emdat["ISO"] != "SPI"] #Remove Canary Islands
emdat = emdat[emdat["ISO"] != "SHN"] #Remove Saint Helena
emdat = emdat[emdat["ISO"] != "IMN"] #Remove Isle of Man

# Remove some variables
dis_df = emdat[["Disaster Type", "Country", "ISO", "Region","Year", "Dis Mag Value", "Dis Mag Scale", "No Affected", "No Homeless", "Start Date", "End Date", "Duration"]].fillna(0)

countries = dis_df["ISO"].unique()
num_countries = len(countries)

regions = dis_df["Region"].unique()
num_regions = len(regions)

Years = temp[(temp["Year"] > 1961) & (temp["Year"] <= 2020)]["Year"]
num_years = len(Years)

In [22]:
# Add noise to 0.5 durations
emdat["Duration"] = emdat["Duration"].clip(lower=0.5)
noise_level = 0.01  # Standard deviation of the Gaussian noise
noise = np.random.normal(0, noise_level, emdat.shape[0])
emdat['Duration'] = np.where(emdat['Duration'] == 0.5, emdat['Duration'] + noise, emdat['Duration'])


In [23]:
outliers_FloodMag, nonoutliers_FloodMag = outlier_seperation(emdat, "Flood", "Dis Mag Value")
outliers_FloodDuration, nonoutliers_FloodDuration = outlier_seperation(emdat, "Flood", "Duration")
outliers_StormDuration, nonoutliers_StormDuration = outlier_seperation(emdat, "Storm", "Duration")
outliers_StormMag, nonoutliers_StormMag = outlier_seperation(emdat, "Storm", "Dis Mag Value")

In [24]:
### PROPORTION OF OUTLIERS BETA DISTRIBUTION

props_Flood = np.zeros((num_regions, num_years))
props_Storm = np.zeros((num_regions, num_years))
for (i,year) in enumerate(Years):
    for (j,region) in enumerate(regions):
        num_total = len(emdat[(emdat["Disaster Type"] == "Flood") & (emdat["Year"] == year) & (emdat["Region"] == region)])
        num_outliers = len(outliers_FloodMag[(outliers_FloodMag["Year"] == year) & (outliers_FloodMag["Region"] == region)])
        if num_total > 0:
            props_Flood[j,i] = num_outliers/num_total
        else:
            props_Flood[j,i] = 0.0
        num_total = len(emdat[(emdat["Disaster Type"] == "Storm") & (emdat["Year"] == year) & (emdat["Region"] == region)])
        num_outliers = len(outliers_StormDuration[(outliers_StormDuration["Year"] == year) & (outliers_StormDuration["Region"] == region)])
        if num_total > 0:
            props_Storm[j,i] = num_outliers/num_total
        else:
            props_Storm[j,i] = 0.0

# Transform the data to be within the open interval (0, 1)
epsilon = 1e-6  # A small constant
props_Flood = np.clip(props_Flood, epsilon, 1 - epsilon)
props_Storm = np.clip(props_Storm, epsilon, 1 - epsilon)

# Initialize arrays to store the estimated parameters
alpha_estimates_flood = np.zeros(num_regions)
beta_estimates_flood = np.zeros(num_regions)
alpha_estimates_storm = np.zeros(num_regions)
beta_estimates_storm = np.zeros(num_regions)

for i in range(num_regions):
    try:
        alpha_est, beta_est, _, _ = beta.fit(props_Flood[i,:], floc=0, fscale=1)
    except:
        print("Couldn't fit flood proportion for region ",regions[i])
        alpha_est = 0
        beta_est = 0
    # Store the estimated parameters
    alpha_estimates_flood[i] = alpha_est
    beta_estimates_flood[i] = beta_est
    try:
        alpha_est, beta_est, _, _ = beta.fit(props_Storm[i,:], floc=0, fscale=1)
    except:
        print("Couldn't fit storm proportion for region ",regions[i])
        alpha_est = 0
        beta_est = 0
    # Store the estimated parameters
    alpha_estimates_storm[i] = alpha_est
    beta_estimates_storm[i] = beta_est

data = []
# Output the results
for region_index in range(num_regions):
    print(f"Region {regions[region_index]}: Flood = (alpha: {round(alpha_estimates_flood[region_index],4)}, beta: {round(beta_estimates_flood[region_index],4)}), Storm = (alpha: {round(alpha_estimates_storm[region_index],4)}, beta: {round(beta_estimates_storm[region_index],4)})")
    data.append({
        "Region": regions[region_index],
        "Alpha_Flood": round(alpha_estimates_flood[region_index], 4),
        "Beta_Flood": round(beta_estimates_flood[region_index], 4),
        "Alpha_Storm": round(alpha_estimates_storm[region_index], 4),
        "Beta_Storm": round(beta_estimates_storm[region_index], 4)
    })

df = pd.DataFrame(data)
df.to_csv("data/model_params/region_proportions_betadist.csv", index=False)    

Couldn't fit flood proportion for region  Northern Africa
Couldn't fit storm proportion for region  Northern Africa
Couldn't fit flood proportion for region  Northern Europe
Couldn't fit storm proportion for region  Western Africa
Couldn't fit storm proportion for region  Russian Federation
Couldn't fit storm proportion for region  Central Asia
Region Southern Asia: Flood = (alpha: 0.1331, beta: 3.3307), Storm = (alpha: 0.1037, beta: 2.4043)
Region Eastern Asia: Flood = (alpha: 0.1174, beta: 1.587), Storm = (alpha: 0.1286, beta: 1.8908)
Region Southern Europe: Flood = (alpha: 0.1033, beta: 33.8577), Storm = (alpha: 0.0709, beta: 0.3681)
Region Western Europe: Flood = (alpha: 0.101, beta: 27.1475), Storm = (alpha: 0.0759, beta: 0.5197)
Region Northern Africa: Flood = (alpha: 0.0, beta: 0.0), Storm = (alpha: 0.0, beta: 0.0)
Region South-Eastern Asia: Flood = (alpha: 0.1071, beta: 13.6267), Storm = (alpha: 0.1408, beta: 1.6379)
Region Eastern Africa: Flood = (alpha: 0.1045, beta: 7.241), 

# Magnitudes and Durations

## Stationary magnitudes and durations

In [25]:
def calculate_gev_aic(y, c, loc, scale):
    """Calculate AIC for GEV model"""
    n = len(y)
    log_likelihood = np.sum(gev.logpdf(y, c, loc=loc, scale=scale))
    k = 3  # Number of parameters (shape, location, scale)
    aic = 2 * k - 2 * log_likelihood
    return aic

def find_distribution(data, var, coeff="1"):
    d = data["Disaster Type"].unique()[0]
    formula = f"Q('{var}') ~ {coeff}"
    rows_to_add = []
    for region in regions:
        region_data = data[data['Region'] == region]
        
        row = None
        aic = np.inf
        poisson_model = None
        
        # Fit the Poisson model
        try:
            poisson_model = glm(formula, data=region_data, family=Poisson()).fit()
            aic = poisson_model.aic
            row = {
                'Region': region,
                'Model': 'Poisson',
                'Parameters': poisson_model.params.to_dict(),
                'AIC': aic,
                'Variable': f"{d}_{var}"
            }
        except Exception as e:
            print(f"Could not fit Poisson model for region {region}: {str(e)}")
        
        # Fit the Negative Binomial model
        if poisson_model is not None:
            overdispersion_factor = poisson_model.deviance / poisson_model.df_resid
            alpha = 1 / overdispersion_factor if overdispersion_factor > 0 else 1  # Avoid division by zero
        else:
            # Iterative process to find alpha
            alpha = 0.5
            try:
                for _ in range(20):  # You can adjust the number of iterations
                    model = glm(formula, data=region_data, family=NegativeBinomial(alpha=alpha)).fit()
                    alpha = 1.0 / (model.pearson_chi2 / model.df_resid)
            except Exception as e:
                print(f"Could not fit NB model for region {region}: {str(e)}")
        try:
            nb_model = glm(formula, data=region_data, family=NegativeBinomial(alpha=alpha)).fit()
            model_params = nb_model.params.to_dict()
            model_params["Alpha"] = alpha
            if nb_model.aic < aic:
                aic = nb_model.aic
                row = {
                    'Region': region,
                    'Model': 'Negative Binomial',
                    'Parameters': model_params,
                    'AIC': aic,
                    'Variable': f"{d}_{var}"
                }
        except Exception as e:
            print(f"Could not fit NB model for region {region}: {str(e)}")

            # Fit the Gamma distribution model
        try:
            gamma_model = glm(formula, data=region_data, family=Gamma(sm.families.links.Log())).fit()
            if gamma_model.aic < aic:
                aic = gamma_model.aic
                scale = gamma_model.scale
                mu = np.exp(gamma_model.params['Intercept'])
                shape = mu/scale
                row = {
                    'Region': region,
                    'Model': 'Gamma',
                    'Parameters': {"Shape":shape, "Scale":scale},
                    'AIC': aic,
                    'Variable': f"{d}_{var}"
                }
        except Exception as e:
            print(f"Could not fit Gamma model for region {region}: {str(e)}")

            
        # Fit the normal distribution model (Gaussian family with an identity link)
        try:
            gaussian_model = glm(formula, data=region_data, family=Gaussian(sm.families.links.Identity())).fit()
            if gaussian_model.aic < aic:
                aic = gaussian_model.aic
                row = {
                    'Region': region,
                    'Model': 'Normal',
                    'Parameters': {
                        **gaussian_model.params.to_dict(),
                        'Scale': gaussian_model.scale
                    },
                    'AIC': aic,
                    'Variable': f"{d}_{var}"
                }
        except Exception as e:
            print(f"Could not fit Gaussian model for region {region}: {str(e)}")
        
        # Fit the GEV model
        try:
            y = region_data[var].dropna()
            c, loc, scale = gev.fit(y)

            # Calculate AIC for GEV
            gev_aic = calculate_gev_aic(y, c, loc, scale)
            if gev_aic < aic:
                aic = gev_aic
                row = {
                    'Region': region,
                    'Model': 'GEV',
                    'Parameters': {'Shape': c, 'Location': loc, 'Scale': scale},
                    'AIC': aic,
                    'Variable': f"{d}_{var}"
                }
        except Exception as e:
            print(f"Could not fit GEV model for region {region}: {str(e)}")

        if row is None:
            row = {
                    'Region': region,
                    'Model': 'Insufficient data',
                    'Parameters': {"Shape":0, "Location":0},
                    'AIC': None,
                    'Variable': f"{d}_{var}"
                }
        rows_to_add.append(row)
        
    return pd.DataFrame(rows_to_add)

In [26]:
rows_to_add = []
globalmu, globalsigma = norm.fit(emdat[emdat["Disaster Type"] == "Storm"]["Dis Mag Value"].dropna())
for r in regions:
    print(r)
    regiondata_outliers = outliers_FloodDuration[outliers_FloodDuration["Region"] == r]["Duration"]
    regiondata_nonoutliers = nonoutliers_FloodDuration[nonoutliers_FloodDuration["Region"] == r]["Duration"]
    regiondata_storms = emdat[(emdat["Disaster Type"] == "Storm") & (emdat["Region"] == r)]["Dis Mag Value"].dropna()
    regiondata_storm_nonoutlier_duration = nonoutliers_StormDuration[nonoutliers_StormDuration["Region"] == r]["Duration"].dropna()
    try:
        params_outliers = weibull_min.fit(regiondata_outliers)
    except:
        print("Could not fit outliers for region ",r)
        params_outliers = (None,None,None)
    params_nonoutliers = gamma.fit(regiondata_nonoutliers)
    mu, sigma = norm.fit(regiondata_storms)
    if np.isnan(mu):
        mu = globalmu
    if np.isnan(sigma):
        sigma = globalsigma
    params_stormduration_nonoutliers = gamma.fit(regiondata_storm_nonoutlier_duration)
    row = {"Region":r, "Shape":params_outliers[0], "Location":params_outliers[1], "Scale":params_outliers[2], "Var":"Flood_outlier_duration"}
    rows_to_add.append(row)
    row = {"Region":r, "Shape":params_nonoutliers[0], "Location":params_nonoutliers[1], "Scale":params_nonoutliers[2], "Var":"Flood_nonoutlier_duration"}
    rows_to_add.append(row)
    row = {"Region":r, "Shape":mu, "Location":None, "Scale":sigma, "Var":"Storm_magnitude"}
    rows_to_add.append(row)
    row = {"Region":r, "Shape":params_stormduration_nonoutliers[0], "Location":params_stormduration_nonoutliers[1], "Scale":params_stormduration_nonoutliers[2], "Var":"Storm_duration_nonoutlier"}
    rows_to_add.append(row)

results = pd.DataFrame(rows_to_add)
results.to_csv("data/model_params/disaster_magnitudes_stationary.csv")

Southern Asia


Eastern Asia
Southern Europe
Western Europe
Northern Africa
South-Eastern Asia
Eastern Africa
Northern Europe
Western Africa
Western Asia
Eastern Europe


/home/jpoulsen/miniconda3/envs/abm/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:2444: RuntimeWarning: invalid value encountered in power
  return np.log(c) + sc.xlogy(c - 1, x) - pow(x, c)


Southern Africa
Middle Africa


/home/jpoulsen/miniconda3/envs/abm/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:419: RuntimeWarning: Mean of empty slice.
  loc = data.mean()
/home/jpoulsen/miniconda3/envs/abm/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/jpoulsen/miniconda3/envs/abm/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:424: RuntimeWarning: Mean of empty slice.
  scale = np.sqrt(((data - loc)**2).mean())


Russian Federation
Central Asia


## Trending Magnitudes and Durations

### Flood Magnitudes

In [27]:
first_year = 1984
last_year = 2020
num_years = last_year - first_year + 1

T = temp[(temp["Year"] >= first_year) & (temp["Year"] <= last_year)].reset_index().drop("index",axis=1)
Years = T["Year"]
T = T["T"]
## CREATE TENSOR ARRAYS FOR FLOOD/STORM COUNTS
T = torch.tensor(T) 

Years = Years[Years > 1984]

In [28]:
def create_datadict(df, var):
    ## CREATE TENSOR ARRAYS FOR FLOOD/STORM COUNTS
    # Initialize a dictionary to store tensors of disaster magnitudes
    data = {}

    #Normalize data
    #floodmag_df = emdat[(emdat["Disaster Type"] == "Flood") & emdat["Dis Mag Value"] > 0].copy()
    mean = df[var].mean()
    std = df[var].std()

    df[var] = (df[var] - mean) / std
    shiftval = abs(df[var].min()) + 1e-5
    df[var] = df[var] + shiftval

    for (i,year) in enumerate(Years[Years>1984]):
        for (j,region) in enumerate(regions):
            dis = df[(df["Year"] == year) & (df["Region"] == region)]
            if len(dis) > 0:
                vals = torch.tensor(dis[var].dropna().values)
            else:
                vals = torch.tensor([])
            data[(region, year)] = vals
    return data, mean, std, shiftval

In [29]:
floodmag_outlier_data, floodmean_outlier, floodstd_outlier, floodshiftval_outlier = create_datadict(outliers_FloodMag, "Dis Mag Value")
floodmag_nonoutlier_data, floodmean_nonoutlier, floodstd_nonoutlier, floodshiftval_nonoutlier = create_datadict(nonoutliers_FloodMag, "Dis Mag Value")

In [30]:
def model_floodmag(T, data_dict):
    with pyro.plate("regions", num_regions):
        shape = pyro.sample("shape", dist.LogNormal(0, 1)) # Assuming shape is positive
        scale_coef = pyro.sample("scale_coef", dist.Normal(0, 1)).unsqueeze(-1)

        # Scale must be positive, model as a function of T
        scale = torch.exp(scale_coef * T) 
    # Sample Weibull parameters for each region
    for (j, region) in enumerate(regions):
        for (i, year) in enumerate(Years):
            magnitudes = data_dict.get((region, year), torch.tensor([]))

            if len(magnitudes) > 0:
                with pyro.plate(f"data_{region}_{year}", magnitudes.shape[0]):
                    pyro.sample(f"obs_{region}_{year}", dist.Weibull(shape[j], scale[j][i]), obs=magnitudes)
    return


In [31]:
# Define guide function
guide =AutoMultivariateNormal(model_floodmag)
# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 500

# Setup the optimizer with learning rate decay
initial_lr = 0.1
gamma_d = 0.001
lrd = gamma_d ** (1 / n_steps)
optimizer = pyro.optim.ClippedAdam({'lr': initial_lr, 'lrd': lrd})

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(model_floodmag, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(T, floodmag_outlier_data)
    if step % 50 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

/home/jpoulsen/miniconda3/envs/abm/lib/python3.9/site-packages/torch/autograd/__init__.py:251: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[0] ELBO: 214.7
[50] ELBO: 170.8
[100] ELBO: 165.9
[150] ELBO: 156.2
[200] ELBO: 153.4
[250] ELBO: 156.9
[300] ELBO: 153.5
[350] ELBO: 152.6
[400] ELBO: 149.5
[450] ELBO: 155.1


In [32]:
predictive = Predictive(model_floodmag, guide=guide, num_samples=1500,
                        return_sites=("shape","scale_coef"))
samples = predictive(T, floodmag_outlier_data)
shape_outliers = samples["shape"].mean(dim=0)
scale_coef_outliers = samples["scale_coef"].mean(dim=0)

In [33]:
# Define guide function
guide =AutoMultivariateNormal(model_floodmag)
# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 500

# Setup the optimizer with learning rate decay
initial_lr = 0.1
gamma_d = 0.001
lrd = gamma_d ** (1 / n_steps)
optimizer = pyro.optim.ClippedAdam({'lr': initial_lr, 'lrd': lrd})

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(model_floodmag, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(T, floodmag_nonoutlier_data)
    if step % 50 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 1169.5
[50] ELBO: 560.4
[100] ELBO: 554.1
[150] ELBO: 546.9
[200] ELBO: 551.3
[250] ELBO: 550.7
[300] ELBO: 548.1
[350] ELBO: 548.9
[400] ELBO: 548.3
[450] ELBO: 548.0


In [34]:
predictive = Predictive(model_floodmag, guide=guide, num_samples=1500,
                        return_sites=("shape","scale_coef"))
samples = predictive(T, floodmag_nonoutlier_data)
shape_nonoutliers = samples["shape"].mean(dim=0)
scale_coef_nonoutliers = samples["scale_coef"].mean(dim=0)

### Storm Duration

In [35]:
stormdur_outlier_data, stormdur_mean_outlier, stormdur_std_outlier, stormdur_shiftval_outlier = create_datadict(outliers_StormDuration, "Duration")

In [36]:
def model_stormduration(T, data_dict):
    with pyro.plate("regions", num_regions):
        shape = pyro.sample("shape", dist.LogNormal(0, 1)) # Assuming shape is positive
        scale_coef = pyro.sample("scale_coef", dist.Normal(0, 1)).unsqueeze(-1)

        # Scale must be positive, model as a function of T
        scale = torch.exp(scale_coef * T) 
    # Sample Weibull parameters for each region
    for (j, region) in enumerate(regions):
        for (i, year) in enumerate(Years):
            magnitudes = data_dict.get((region, year), torch.tensor([]))

            if len(magnitudes) > 0:
                with pyro.plate(f"data_{region}_{year}", magnitudes.shape[0]):
                    pyro.sample(f"obs_{region}_{year}", dist.Weibull(shape[j], scale[j][i]), obs=magnitudes)
    return


In [37]:
# Define guide function
guide =AutoMultivariateNormal(model_stormduration)
# Reset parameter values
pyro.clear_param_store()

In [38]:
# Define the number of optimization steps
n_steps = 500

# Setup the optimizer with learning rate decay
initial_lr = 0.01
gamma_d = 0.01
lrd = gamma_d ** (1 / n_steps)
optimizer = pyro.optim.ClippedAdam({'lr': initial_lr, 'lrd': lrd})

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=1)
svi = SVI(model_stormduration, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(T, stormdur_outlier_data)
    if step % 50 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 313.4
[50] ELBO: 192.7
[100] ELBO: 197.6
[150] ELBO: 185.8
[200] ELBO: 180.6
[250] ELBO: 179.2
[300] ELBO: 183.4
[350] ELBO: 179.5
[400] ELBO: 177.9
[450] ELBO: 181.1


In [39]:
predictive = Predictive(model_stormduration, guide=guide, num_samples=1500,
                        return_sites=("shape","scale_coef"))
samples = predictive(T, stormdur_outlier_data)
shape_stormduration = samples["shape"].mean(dim=0)
scale_coef_stormduration = samples["scale_coef"].mean(dim=0)

In [40]:
storm_duration = pd.DataFrame({"Region":regions, "Shape":shape_stormduration, "Scale_coef":scale_coef_stormduration, "Var":"Storm_duration_outlier"})
storm_duration.loc[len(storm_duration.index)] = {"Region":"Global", "Shape":stormdur_mean_outlier, "Scale_coef":0, "Var":"Storm_duration_outliers_scalermean"}
storm_duration.loc[len(storm_duration.index)] = {"Region":"Global", "Shape":stormdur_std_outlier, "Scale_coef":0, "Var":"Storm_duration_outliers_scalerstd"}
storm_duration.loc[len(storm_duration.index)] = {"Region":"Global", "Shape":stormdur_shiftval_outlier, "Scale_coef":0, "Var":"Storm_duration_outliers_scalershiftval"}

floodmag_outliers = pd.DataFrame({"Region":regions, "Shape":shape_outliers, "Scale_coef":scale_coef_outliers, "Var":"Flood_magnitude_outlier"})
floodmag_outliers.loc[len(floodmag_outliers.index)] = {"Region":"Global", "Shape":floodmean_outlier, "Scale_coef":0, "Var":"Flood_magnitude_outliers_scalermean"}
floodmag_outliers.loc[len(floodmag_outliers.index)] = {"Region":"Global", "Shape":floodstd_outlier, "Scale_coef":0, "Var":"Flood_magnitude_outliers_scalerstd"}
floodmag_outliers.loc[len(floodmag_outliers.index)] = {"Region":"Global", "Shape":floodshiftval_outlier, "Scale_coef":0, "Var":"Flood_magnitude_outliers_scalershiftval"}

floodmag_nonoutliers = pd.DataFrame({"Region":regions, "Shape":shape_nonoutliers, "Scale_coef":scale_coef_nonoutliers, "Var":"Flood_magnitude_nonoutlier"})
floodmag_nonoutliers.loc[len(floodmag_nonoutliers.index)] = {"Region":"Global", "Shape":floodmean_nonoutlier, "Scale_coef":0, "Var":"Flood_magnitude_nonoutliers_scalermean"}
floodmag_nonoutliers.loc[len(floodmag_nonoutliers.index)] = {"Region":"Global", "Shape":floodstd_nonoutlier, "Scale_coef":0, "Var":"Flood_magnitude_nonoutliers_scalerstd"}
floodmag_nonoutliers.loc[len(floodmag_nonoutliers.index)] = {"Region":"Global", "Shape":floodshiftval_nonoutlier, "Scale_coef":0, "Var":"Flood_magnitude_nonoutliers_scalershiftval"}

In [41]:
results = pd.concat([storm_duration,floodmag_outliers,floodmag_nonoutliers], ignore_index=True)
results.to_csv("data/model_params/disaster_magnitudes_trending.csv")

In [42]:
results

,Region,Shape,Scale_coef,Var
0,Southern Asia,0.880171,0.188470,Storm_duration_outlier
1,Eastern Asia,0.653471,-0.045347,Storm_duration_outlier
2,Southern Europe,0.696473,-0.098530,Storm_duration_outlier
3,Western Europe,0.500165,-0.169095,Storm_duration_outlier
4,Northern Africa,1.024065,0.012882,Storm_duration_outlier
5,South-Eastern Asia,0.726213,0.127292,Storm_duration_outlier
6,Eastern Africa,0.970135,-0.143109,Storm_duration_outlier
7,Northern Europe,0.507233,-0.216000,Storm_duration_outlier
8,Western Africa,1.628708,0.576802,Storm_duration_outlier
9,Western Asia,1.280640,-0.153490,Storm_duration_outlier


# Disaster Occurence Rates

In [43]:
X_s = pd.merge(dis_df,popsize, on=["Year","ISO"],how="left")
X_s = pd.merge(X_s,temp, on="Year", how="left")
X_s["Homeless_pct"] = X_s["No Homeless"] / X_s["Pop"]
X_s=X_s.dropna()
#X = pd.get_dummies(X, columns=['Region'])
# Convert all boolean columns in the DataFrame to integers (0 or 1)
bool_columns = X_s.select_dtypes(include=['bool']).columns
X_s[bool_columns] = X_s[bool_columns].astype(int)

# REMOVE REDUNDANT VARIABLES
# We remove all indicators that are in numbers and stick to the ones in percentages, as they are otherwise the same
X_s = X_s.drop(["Dis Mag Scale", "No Affected","No Homeless", "Start Date", "End Date", "Size", "Pop"],axis=1)

In [44]:
# Group the data and get counts for each disaster type per year per ISO code
occurences_df = X_s.pivot_table(index=['ISO', 'Country', 'Year'], 
                                   columns='Disaster Type', 
                                   aggfunc='size', 
                                   fill_value=0).reset_index()

# Rename the columns for clarity
occurences_df.columns = ['ISO', 'Country', 'Year', 'Floods', 'Storms']
occurences_df.head()

,ISO,Country,Year,Floods,Storms
0,AFG,Afghanistan,1963,1,0
1,AFG,Afghanistan,1972,1,0
2,AFG,Afghanistan,1976,1,0
3,AFG,Afghanistan,1978,1,0
4,AFG,Afghanistan,1980,1,0


In [45]:
flood_counts = torch.zeros(num_countries, num_years)
storm_counts = torch.zeros(num_countries, num_years)

for (i,year) in enumerate(Years):
    for (j,country) in enumerate(countries):
        vals = occurences_df[(occurences_df["Year"] == year) & (occurences_df["ISO"] == country)]
        if len(vals) > 0:
            floods = vals["Floods"].iloc[0]
            storms = vals["Storms"].iloc[0]
        else:
            floods = 0
            storms = 0
        flood_counts[j,i] = floods
        storm_counts[j,i] = storms 

In [46]:
## MODEL for determining latent variables for occurence rate as a function of temperature

def model(T, flood_counts, storm_counts):
    T = T.unsqueeze(0)
    with pyro.plate("countries", num_countries):
        alpha_f = pyro.sample("alpha_f", dist.Normal(0, 1)).unsqueeze(-1) # Prior for the intercept
        beta_f = pyro.sample("beta_f", dist.Normal(0, 1)).unsqueeze(-1) # Prior for the slope
        alpha_s = pyro.sample("alpha_s", dist.Normal(0, 1)).unsqueeze(-1)  # Prior for the intercept
        beta_s = pyro.sample("beta_s", dist.Normal(0, 1)).unsqueeze(-1) # Prior for the slope

        # Expand T to match the number of countries
        T_expanded = T.expand(num_countries, -1)
        # Calculate lambda_f and lambda_s for each country for each year
        lambda_f = torch.exp(alpha_f + beta_f * T_expanded)
        lambda_s = torch.exp(alpha_s + beta_s * T_expanded)

    with pyro.plate("years", num_years,dim=-1):
        pyro.sample("floods", dist.Poisson(lambda_f), obs=flood_counts)
        pyro.sample("storms", dist.Poisson(lambda_s), obs=storm_counts)

    return


In [47]:
# Run inference in Pyro
nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=100, num_chains=1)
mcmc.run(T, flood_counts, storm_counts)

Warmup:   0%|          | 0/1100 [00:00, ?it/s]

Sample: 100%|██████████| 1100/1100 [05:29,  3.34it/s, step size=9.84e-02, acc. prob=0.849]


In [48]:
# Extract samples from posterior
posterior_samples = mcmc.get_samples()

alpha_f = posterior_samples['alpha_f'].mean(dim=0)
beta_f = posterior_samples['beta_f'].mean(dim=0)
alpha_s = posterior_samples['alpha_s'].mean(dim=0)
beta_s = posterior_samples['beta_s'].mean(dim=0)

In [49]:
results = pd.DataFrame({"Country":countries, "Alpha_flood_occurences":alpha_f, "Beta_flood_occurences":beta_f, "Alpha_storm_occurences":alpha_s, "Beta_storm_occurences":beta_s })
results.to_csv("data/model_params/occurences_countries_trending.csv")

# Displacement

Linear regression models for percentage of displacement

In [50]:
X_s = pd.merge(dis_df,popsize, on=["Year","ISO"],how="left")
X_s = pd.merge(X_s,temp, on="Year", how="left")
X_s["Homeless_pct"] = X_s["No Homeless"] / X_s["Pop"]
X_s=X_s.dropna()
#X = pd.get_dummies(X, columns=['Region'])
# Convert all boolean columns in the DataFrame to integers (0 or 1)
bool_columns = X_s.select_dtypes(include=['bool']).columns
X_s[bool_columns] = X_s[bool_columns].astype(int)

# REMOVE REDUNDANT VARIABLES
# We remove all indicators that are in numbers and stick to the ones in percentages, as they are otherwise the same
X_s = X_s.drop(["Dis Mag Scale", "No Affected", "Start Date", "End Date", "ISO", "Country", "Year", "T", "Region"],axis=1)
X_s = X_s[X_s["Homeless_pct"] > 0]
X_s = X_s[(X_s["Dis Mag Value"] > 0) & (X_s["Duration"] > 0)]

In [51]:
X_f = X_s[X_s["Disaster Type"] == "Flood"].drop("Disaster Type",axis=1)
X_s = X_s[X_s["Disaster Type"] == "Storm"].drop("Disaster Type",axis=1)

In [52]:
# Define the predictor variables (features) and the target variable
X = X_s[['Dis Mag Value', 'Duration', 'Popdens']]
y = X_s['Homeless_pct']


# Create and fit the linear regression model
model = LinearRegression()
model.fit(X, y)

storm_model_pct = {"Model":"Storm_disp_pct","Intercept":model.intercept_, "Dis Mag Value":model.coef_[0], "Duration":model.coef_[1], "Popdens":model.coef_[2], "rmse":0.0001}

# Define the predictor variables (features) and the target variable
X = X_s[['Dis Mag Value', 'Duration', 'Popdens', 'Pop']]
y = X_s['No Homeless']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit Linear regression to training data
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Calculating rmse
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Refit to all datapoints
model = LinearRegression()
model.fit(X, y)

storm_model_abs = {"Model":"Storm_disp_abs","Intercept":model.intercept_, "Dis Mag Value":model.coef_[0], "Duration":model.coef_[1], "Popdens":model.coef_[2], "Pop":model.coef_[3], "rmse":rmse}

In [53]:
# Define the predictor variables (features) and the target variable
X = X_f[['Dis Mag Value', 'Duration', 'Popdens']]
y = X_f['Homeless_pct']


# Create and fit the linear regression model
model = LinearRegression()
model.fit(X, y)

flood_model_pct = {"Model":"Flood_disp_pct","Intercept":model.intercept_, "Dis Mag Value":model.coef_[0], "Duration":model.coef_[1], "Popdens":model.coef_[2], "rmse":0.0001}

# Define the predictor variables (features) and the target variable
X = X_f[['Dis Mag Value', 'Duration', 'Popdens', 'Pop']]
y = X_f['No Homeless']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit Linear regression to training data
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Calculating rmse
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Refit to all datapoints
model = LinearRegression()
model.fit(X, y)

flood_model_abs = {"Model":"Flood_disp_abs","Intercept":model.intercept_, "Dis Mag Value":model.coef_[0], "Duration":model.coef_[1], "Popdens":model.coef_[2],"Pop":model.coef_[3], "rmse":rmse}

In [54]:
regmodels = pd.DataFrame([storm_model_pct,storm_model_abs,flood_model_pct,flood_model_abs])
regmodels.to_csv("data/model_params/displacement_regression.csv")

# Indicies

In [55]:
countries_withnowri = ["HKG", "TWN", "MAC", "PSE"]
countries = countries[~np.isin(countries, countries_withnowri)]

In [56]:
country_regions = []
for c in countries:
    dataslice = emdat[emdat["ISO"] == c]
    country_regions.append(dataslice["Region"].unique()[0])

In [57]:
country_region = pd.DataFrame({"Country":countries, "Region":country_regions})
country_region.to_csv("data/model_params/countries_regions.csv")